In [27]:
import pandas as pd
import numpy as np
from colormath.color_objects import sRGBColor, LCHuvColor, LCHabColor, HSVColor
from colormath.color_conversions import convert_color
from PIL import Image
import PIL as pil
import matplotlib.pyplot as plt
import matplotlib.colors
import time
import math

In [28]:
def plot_color_hsv(h, s, v):
    s = 1.0 # for checking hues
    v = 1.0
    r, g, b = matplotlib.colors.hsv_to_rgb((h,s,v))
    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.set_facecolor((r,g,b))
    plt.show()

In [29]:
cluster_csv_fp = "Data\\fine_clusters_dbscan_eps=10_minsamples=32.csv"
df_clusters = pd.read_csv(cluster_csv_fp, delimiter=";")
df_clusters["colorname"] = None
df_clusters.head()

,Unnamed: 0,id,platform,hashtag,r,g,b,h,s,v,relative_frequency,colorname
0,0,7172368921318657322,tiktok,climatechange,0.952941,0.565785,0.063529,0.094118,0.933333,0.952941,0.006185,None
1,0,7172368921318657322,tiktok,climatechange,0.678431,0.417435,0.243437,0.066667,0.641176,0.678431,0.069010,None
2,0,7172368921318657322,tiktok,climatechange,0.776471,0.133979,0.270036,0.964706,0.827451,0.776471,0.004774,None
3,0,7172368921318657322,tiktok,climatechange,0.588235,0.466870,0.348328,0.082353,0.407843,0.588235,0.013238,None
4,0,7172368921318657322,tiktok,climatechange,0.996078,0.187497,0.358726,0.964706,0.811765,0.996078,0.007487,None


In [30]:
def get_colorname_from_lch(l,c,h):
    # achromatic colors
    if l < 10:
        return "black"
    if l > 90:
        return "white"
    if c < 10:
        return "gray"
    
    # now that all the achromatic colors are out of the way,
    # we can check for chromatic colors using hue
    # this is the same as Yu et al. (2020)
    if h < 30:
        return "orange"
    if h < 60:
        return "orange-yellow"
    if h < 90:
        return "yellow"
    if h < 120:
        return "yellow-green"
    if h < 150:
        return "green"
    if h < 180:
        return "green-blue"
    if h < 210:
        return "blue"
    if h < 240:
        return "blue-violet"
    if h < 270:
        return "violet"
    if h <= 300:
        return "violet-red"
    if h <= 330:
        return "red"
    else:
        return "red-orange"
    
    chromatic_colors = ["orange", "orange_yellow", "yellow", "yellow-green", "green", "green-blue", "blue", "blue-violet", "violet", "violet-red", "red", "red-orange"]
    return chromatic_colors(int(math.floor( h % 30 )))
    

In [31]:
df_clusters["colorname"] = np.NaN
visualize = False

for idx, row in df_clusters.iterrows():
    hue, sat, val = row["h"], row["s"], row["v"]
    red, green, blue = row["r"], row["g"], row["b"]
    color_hsv = HSVColor(hue, sat, val)
    color_rgb = sRGBColor(red, green, blue)
    color_lch = convert_color(color_rgb, LCHuvColor)
    l, c, h = color_lch.lch_l, color_lch.lch_c, color_lch.lch_h
    colorname = get_colorname_from_lch(l, c, h)
    df_clusters["colorname"][idx] = colorname
    if visualize:
        print(f"This color is {l}, {c}, {h} in LCh color space.")
        print(f"This color is {hue}, {sat}, {val} in HSV color space.")
        print(f"This color is {colorname.upper()}")
        plot_color_hsv(hue,sat,val)

print(df_clusters.head())
output_path = "Data\\fine_clusters_dbscan_eps=10_minsamples=32_with_colornames.csv"
df_clusters.to_csv(output_path, sep=";")
    

    

    


C:\Users\Maxim\AppData\Local\Temp\ipykernel_39216\41020486.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clusters["colorname"][idx] = colorname


   Unnamed: 0                   id platform        hashtag         r  \
0           0  7172368921318657322   tiktok  climatechange  0.952941   
1           0  7172368921318657322   tiktok  climatechange  0.678431   
2           0  7172368921318657322   tiktok  climatechange  0.776471   
3           0  7172368921318657322   tiktok  climatechange  0.588235   
4           0  7172368921318657322   tiktok  climatechange  0.996078   

          g         b         h         s         v  relative_frequency  \
0  0.565785  0.063529  0.094118  0.933333  0.952941            0.006185   
1  0.417435  0.243437  0.066667  0.641176  0.678431            0.069010   
2  0.133979  0.270036  0.964706  0.827451  0.776471            0.004774   
3  0.466870  0.348328  0.082353  0.407843  0.588235            0.013238   
4  0.187497  0.358726  0.964706  0.811765  0.996078            0.007487   

       colorname  
0  orange-yellow  
1  orange-yellow  
2         orange  
3  orange-yellow  
4         orange  
